In [9]:
%matplotlib inline

import mido
import numpy as np
from math import ceil
import keras
import random
import music21
import glob
import os

In [29]:
def transpose(score, newkey='C'):
    """ Transposes the midi file to C. 
    Args: music21.stream.Score
    Returns: music21.stream.Score
    """

    keys = {'C':0, 
            'C#':1, 
            'D-':1,
            'D':2,
            'D#':3,
            'E-':3,
            'E':4,
            'F-':4,
            'E#':5,        
            'F':5,
            'F#':6,
            'G-':6,
            'G':7,
            'G#':8,
            'A-':8,
            'A':9,
            'A#':10,
            'B-':10,
            'B':11,
            'B#':0,
           }

    oldkey = score.analyze('key')

    if oldkey.mode == "major":
        halfSteps = keys[oldkey.tonic.name.upper()]

    elif oldkey.mode == "minor":
        halfSteps = keys[oldkey.tonic.name.upper()] + 4

    halfSteps %= 12

    newscore = score.transpose(-halfSteps)
    trans_key = newscore.analyze('key')
    print oldkey, trans_key
    return newscore

In [3]:
twin = mido.MidiFile('data/twinkle.mid')
twin2 = mido.MidiFile('data/twinkle_twinkle.mid')
simp = mido.MidiFile('data/simple_test.mid')
simp2 = mido.MidiFile('data/simple_test_2notes.mid')
song = mido.MidiFile('data/bags_groove_jh.mid')
mond = mido.MidiFile('data/mond_3.mid')
abba = mido.MidiFile('data/ABBA_-_Dancing_Queen.mid')
bach = mido.MidiFile('data/bach_variations_sn.mid')
cop = mido.MidiFile('data/appspg13.mid')
chords = mido.MidiFile('deeplearningnet/data/Nottingham/train/ashover_simple_chords_1.mid')

### Format

midi -> notelist -> music_mat -> music_mat -> notelist -> midi

In [3]:
s = music21.converter.parse('data/twinkle.mid')

### Create the notelist (note, start, duration)

In [4]:
# creates notelist - a list for all the notes

midifile = chords

# note_list = (note, start, duration)
# have to address if notes are not perfectly inside a tick_step

resolution = 8 # eighth notes

# this gives # of ticks in a column of the piano sheet
tick_step = round(float(midifile.ticks_per_beat/resolution))

note_range = (20, 90)
active_notes = [0] * abs(np.diff(note_range)[0])

counter = 0
notelist = []

rawmsgs = [msg for track in midifile.tracks for msg in track]

# have to reset counter when a new track is detected
for msg in rawmsgs:
    if msg.type not in set(['note_on','note_off']):
        continue
    
    #counter += ceil(msg.time/tick_step)
    counter += msg.time
    
    if msg.type == 'note_on' and msg.velocity > 0:        
        active_notes[msg.note - note_range[0]] = counter
        
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        # fill everything up to this with 1s
        start = active_notes[msg.note - note_range[0]]  #round this section
               
        notelist.append((msg.note, int(ceil(start/tick_step)), int(ceil((counter-start)/tick_step))))
        active_notes[msg.note - note_range[0]] = 0

notelist = sorted(notelist, key=lambda x: x[1])

In [5]:
steps = int(ceil(counter/tick_step))
music_mat = np.zeros([steps, int(np.diff(note_range))])

### Fill the music_mat

In [6]:
cnt = 0
for nt in notelist:
    c = range(nt[1], (nt[2] + nt[1]))
    r = [nt[0] - note_range[0]] * len(c)
    music_mat[c,r] = np.array([1] * len(r) )
    cnt += 1

In [7]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

### Slicing the matrix into X,y into a 3D matrix

In [8]:
maxlen = 50 * 2
nnotes = np.diff(note_range)[0]
step = 1

In [9]:
X= []
y = []
for ix in range(0, steps-maxlen-1, step):
    end = ix + maxlen
    X.append(music_mat[ix:end:1, :])
    y.append(music_mat[end+1,:])
X = np.stack(X)
y = np.stack(y)

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, nnotes)))
model.add(Dense(nnotes))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [12]:
loss = []
for _ in range(10):
    res = model.fit(X, y, batch_size=128, nb_epoch=1)
    loss.extend(res.history['loss'])

Epoch 1/1
1391/1391 [==============================] - 3s - loss: 8.2900     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 7.0379     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 6.4306     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 5.9572     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 5.6943     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 5.5403     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 5.1264     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 5.0427     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 4.9516     
Epoch 1/1
1391/1391 [==============================] - 0s - loss: 4.9501     


### Generating new tunes

In [13]:
tsteps = 100

start_ix = random.randint(0, steps-maxlen - 1)
psg_seed = music_mat[start_ix:start_ix+maxlen,:]
x = psg_seed[None,:,:]

In [14]:
new_song = []
new_song.append(x)

for _ in range(tsteps):

    preds = model.predict(x, verbose=0)[0]

    # this is a timestep slice
    new_note = np.zeros(nnotes)
    new_note[preds.argmax()] = 1
    new_note = new_note[None, None,:]

    # redo the seed psg
    psg = np.concatenate([x,new_note], axis=1)
    x = psg[:, 1:,:]

    new_song.append(new_note)

new_song = np.concatenate(new_song, axis=1)
new_song = np.squeeze(new_song)

### Song rolls to midi

In [59]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
mid.tracks.append(track)

### Music mat to notelist

In [60]:
active_notes = np.array([0] * abs(np.diff(note_range)[0]))

In [71]:
track = MidiTrack()
nsteps = new_song.shape[0]
last_event = 0
for ix in range(nsteps):
    step_slice = new_song[ix,:]
    
    diff = [ix for ix,(x,y) in enumerate(zip(step_slice, active_notes)) if x!=y]
    if len(diff) == 0:
        last_event += 1
        continue

    # this means there is a difference
    for note_ix in diff:
        # off to on
        note = note_ix+min(note_range)
        if active_notes[note_ix] == 0:
            track.append(Message('note_on', note=note, velocity=127, time=int(tick_step*last_event) ))
            active_notes[note_ix] = 1
        else:
            track.append(Message('note_off', note=note, velocity=127, time=int(tick_step*last_event)))
            active_notes[note_ix] = 0
        last_event = 0        

### Save to file

In [75]:
# save the file
mid = MidiFile()
mid.tracks.append(track)
mid.save('new_song.mid')

In [116]:
test[np.array([0,2])] = 3

In [72]:

coords = np.nonzero(new_song)
track = MidiTrack()

time = 0
for ix,note in zip(*coords):
    if ix == note:
        track.append(Message('note_on', note=note+min(note_range), velocity=127, time=0))
        active_notes[note] = True
    elif ix != note:
        active_idx = [idx for idx,y in enumerate(active_notes) if y]

        if len(active_idx) > 0:
            track.append(Message('note_off', note=active_idx[0], velocity=127, time=int((ix-time)*tick_step)))    
        for idx in active_idx[1:]:
            track.append(Message('note_off', note=idx, velocity=127, time=0))
        
        active_notes = [False] * abs(np.diff(note_range)[0])
        track.append(Message('note_on', note=note, velocity=127, time=0))
        active_notes[note] = True
    time = ix